# Section 3
## Regularization

We previously looked at the effect of overfitting, and how we can detect it using cross-validation. Detecting overfitting is good, but what about preventing it from happening? Overfitting occurs when we have too many parameters relative to the amount of data we have. How can we modulate our model parameters such that overfitting is limited?  
  
The answer is _regularization_. Regularization is a penalty applied directly to model parameters _independent_ of the model performance. Without regularization, models are optimized relative to a particular metric. For example, the squared error:  
\begin{equation}
    Err(Y, \hat{Y}) = \sum |Y-\hat{Y}|^2
\end{equation}

The metric relies only on predictions and labels. We can add a second term that applies only on the model parameters, `w`:  

\begin{equation}
    Err(Y, \hat{Y}) = \sum (Y-\hat{Y})^2 + \alpha w^T w  \\
    = \sum |Y-\hat{Y}|^2 + \alpha \sum w_i^2  \\
    = (Prediction Loss) + (Regularization Loss)
\end{equation}

The second term is the regularization term: the it increases with the square of the model parameters. Intuitively, this means that large model parameters are discouraged, while a mix of smaller parameters is preferred. This form of regularization is 'L2 regularization.' The weighting coefficient, $\alpha$, determines the balance between regularization

## Regularization Example 1 - L2
Let's look at the effect of regularization on an increasing model complexity. In the first section, we saw how increasing model complexity via `PolynomialFeatures` resulted in overfitting of our model. Let's examine the effect of increasing the degree on `PolynomialFeatures` while using regularization.  
In this example, we'll generate a noisy parabola, then try to fit polynomials of increasing complexity. In `sklearn`, regularization is often enabled via the `penalty` argument. In the case of linear regression, it is implemented in a separate class. For L2 regularization, we use `Ridge`, in the same way we previously used `LinearRegression`.

In [ ]:
# Generate 
import numpy as np
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.linear_model import Ridge

x_data = np.random.random((200,1))
y_label = (x_data-0.2)*(x_data-0.6) + np.random.randn(x_data.shape[0], 1)*0.05
x_train, x_test, y_train, y_test = train_test_split(x_data, y_label, train_size=0.4)
plt.plot(x_data, y_label,'.')

In [ ]:
# Import
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
# Create polynomials of features
###
poly_feat = PolynomialFeatures(degree=4)
feat_train = poly_feat.fit_transform(x_train)
###

# Fit model to the new features

# Create polynomials of features for test set
# Evaluate on test set
feat_test = poly_feat.fit_transform(x_test)

linear_test_scr_list = []
linear_train_scr_list = []
linear_coef_mag_list = []  # Let's look at the regularization error

ridge_test_scr_list = []
ridge_train_scr_list = []
ridge_coef_mag_list = []

for i in range(20):  ############ You can modify this value here
    poly_feat = PolynomialFeatures(degree=i)
    feat_train = poly_feat.fit_transform(x_train)
    
    # Linear model
    linear_mdl = LinearRegression(fit_intercept=False)
    linear_mdl.fit(feat_train, y_train)
    
    # Ridge
    ########## Modify the alpha parameter to modulate the relative importance of the prediction error
    ########## vs. regularization
    ridge_mdl = Ridge(fit_intercept=False, alpha=1)     
    ridge_mdl.fit(feat_train, y_train)
    
    feat_test = poly_feat.fit_transform(x_test)
    
    linear_train_scr_list.append(linear_mdl.score(feat_train, y_train))
    linear_test_scr_list.append(linear_mdl.score(feat_test, y_test))
    linear_coef_mag_list.append(linear_mdl.coef_ @ linear_mdl.coef_.T)
    
    ridge_train_scr_list.append(ridge_mdl.score(feat_train, y_train))
    ridge_test_scr_list.append(ridge_mdl.score(feat_test, y_test))
    ridge_coef_mag_list.append(np.squeeze(ridge_mdl.coef_ @ ridge_mdl.coef_.T))


In [ ]:
# Score plot
plt.figure()
plt.subplot(1,2,1)
plt.plot(linear_train_scr_list, 'b.-')
plt.plot(linear_test_scr_list, 'r.-')
plt.xlabel('Degree fit')
plt.ylabel('R2 Score')
plt.legend(['Train Score', 'Test Score'])
plt.title('Score vs. Degree (linear)')
plt.subplot(1,2,2)
plt.plot(ridge_train_scr_list, 'bx-')
plt.plot(ridge_test_scr_list, 'r.-')
plt.xlabel('Degree fit')
plt.ylabel('R2 Score')
plt.legend(['Train Score', 'Test Score'])
plt.title('Score vs. Degree (reg.)')

# Prediction plot
plt.figure()
plt.subplot(1,2,1)
pred = linear_mdl.predict(feat_test)
xind = np.argsort(x_test[:,0])
plt.plot(x_test[xind], pred[xind],'r.-')
plt.plot(x_test, y_test, 'b.')
plt.xlabel('feature value')
plt.ylabel('label value')
plt.legend(['Labels','Prediction'])
plt.title('Linear regression')

plt.subplot(1,2,2)
pred = ridge_mdl.predict(feat_test)
xind = np.argsort(x_test[:,0])
plt.plot(x_test[xind], pred[xind],'r.-')
plt.plot(x_test, y_test, 'b.')
plt.xlabel('feature value')
plt.ylabel('label value')
plt.legend(['Labels','Prediction'])
plt.title('Linear regression (reg.)')

plt.figure()
#plt.plot(linear_coef_mag_list, 'k.-')  # Uncomment this to show linear model coefficient magnitude
plt.plot(ridge_coef_mag_list, 'b.-')
#plt.legend(['Linear', 'Ridge'])
plt.title('Model Param Magnitude vs. Complexity')



Examine the four plots and look at the differences between the non-regularized and regularized models. How does the performance differ with increasing model complexity? What about the predicted curve?  
  
Return to the Ridge initialization and modify the value of `alpha`, the weighting parameter for regularization. What is the effect of increasing the value? What is the effect of decreasing it?

## Regularization Example 2 - L1  
We saw the effect of L2 regularization in the previous example. Let's examine the effect of L1 regularization, implemented in `sklearn.linear_model.Lasso`. L1 regularization is implemented as:
\begin{equation}
Err(Y, \hat(Y)) = \sum (Y-\hat{Y})^2 + \alpha \sum |w|
\end{equation}
The model parameters are penalized by the sum of the absolute value instead of the sum of squares. Intuitively, this results in a sparse representation; parameters that do not reduce the error proportional to their weight are removed.

Instead of reproducing all of the previous plots, let's instead look at the parameter values directly. Using the data and `PolynomialFeatures` from above:  
  
1) Initialize and fit two models: Ridge and Lasso.  
2) Print the model parameter magnitude (sum of squares of the coefficients) for degree 20.  
3) Use a bar plot to compare parameter values across the polynomials. What difference do you see between the two?  
  
Note: There's a quirk with ridge.coef_; use np.squeeze(ridge.coef_) to get it into the expected shape.

In [ ]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

# Create data
x_data = np.random.random((500,1))
y_label = (x_data-0.2)*(x_data-0.6) + np.random.randn(x_data.shape[0], 1)*0.01
x_train, x_test, y_train, y_test = train_test_split(x_data, y_label, train_size=0.4)

# Create polynomials of features
poly_feat = PolynomialFeatures(degree=20)
feat_train = poly_feat.fit_transform(x_train)
feat_test = poly_feat.fit_transform(x_test)

# Initialize models
ridge = Ridge(fit_intercept=False, alpha=0.1)  ### Vary alpha
lasso = Lasso(fit_intercept=False, alpha=1e-6, max_iter=10000, tol=2e-3)  ### Vary alpha

ridge.fit(feat_train, y_train)
lasso.fit(feat_train, y_train)

# Print magnitudes
lasso_w = lasso.coef_
print('L1 (Lasso) magnitudes: ' + str(lasso_w @ lasso_w.T))
print('L1 score: ' + str(lasso.score(feat_test, y_test)))

ridge_w = np.squeeze(ridge.coef_)
print('L2 (Ridge) magnitude: ' + str(ridge_w @ ridge_w.T))
print('L2 score: ' + str(ridge.score(feat_test, y_test)))

# Bar plot
plt.figure()
plt.subplot(2,1,1)
plt.bar(np.arange(len(lasso_w)), lasso_w)
plt.title('L1 Regularized')
plt.subplot(2,1,2)
plt.bar(np.arange(len(ridge_w)), ridge_w)
plt.title('L2 Regularized')
plt.xlabel('Degree')

Note the difference between the two distributions; what is the effect of L1 regularization on the parameter values? When would you prefer one over the other?

## Hyperparameters
In the regularization examples, we saw the coefficient $\alpha$ that determined how much our model parameters should be penalized relative to the prediction error. This type of parameter is not directly part of our model: it is not modified by our data, but it does directly affect our model's ability to generalize.  
  
In the example of logistic regression, we specified `solver`, which dictates which implementation will be used to determine the optimal parameters. Different implementations may not converge to a solution under certain conditions, which in turn would give us a poor model.  
  
External parameters that are not modified by data are called, 'hyperparameters'. Common ones include regularization weights, learning rate, or sample weights. The 'best' values for hyperparameters are determined via cross-validation.

## Model Interpretation

Examine the model parameters in the last example, and compare them to the parameters for the generating function: $x^2 - 0.8x + 0.12$. Both models get in the general area of the parameter values: small positive intercept, negative weight for the linear term, and a positive weight for the quadratic term.  
Neither of them is _quite_ right, though. They both have a cubic term, and the other terms aren't quite large enough.  
  
How would we interpret this model? Can we say that $x$ is twice as important as $x^2$ if its coefficient is twice as large?  
  
This is where machine learning and scientists might diverge: whereas machine learning may not care about what the model is so long as it performs well, scientists are interested in understanding what the model is doing; we are interested in _causality_. We want to know a feature's influence on the output. However, if we can't control our inputs, we can't really infer causality.  
  
We can make a slightly different statement: instead of, "this feature causes X," we can say, "this feature is useful for predicting X." It's the same distinction as correlation vs. causation.  
  
The distinction might be unpleasant, but we don't need to toss out the entirety of machine learning. We don't need one approach to solve every part of the problem. We can use our models to identify features of interest. We can have observational data, use ML models to identify useful features, then return to our experiments and perform causal experiments.

## Feature Selection
We saw in the regularization example that L1 regularization produces a _sparse_ representation; i.e., it implicitly reduces the number of non-zero coefficients. We might instead be interested in explicitly reducing the number of features, or simply seeing which features are most important for prediction performance.  
  
`sklearn` has a number of tools for feature selection ([link](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_selection)). We'll take a look at recursive feature elimination (RFE). It first fits your model to all of your input features, then fits all but of your input features, etc. The features which contribute the most to improving prediction are kept until the end.

### Feature Selection Exercise
In this example, we'll examine the models that we saw in the regularization and see the performance of RFE.  
  
1) Generate data.  
2) Augment the data using `PolynomialFeatures`  
3) Initialize three models: LinearRegression, Lasso, Ridge  
4) Use RFE to identify which features are important for each model. Print the rankings of each.  
  
Note: RFE stores rankings in rfe.ranking_

In [ ]:
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import RFE
import numpy as np

# Create data
x_data = np.random.random((500,1))
y_label = (x_data-0.2)*(x_data-0.6) + np.random.randn(x_data.shape[0], 1)*0.01
x_train, x_test, y_train, y_test = train_test_split(x_data, y_label, train_size=0.4)

# Create polynomials of features
poly_feat = PolynomialFeatures(degree=20)
feat_train = poly_feat.fit_transform(x_train)
feat_test = poly_feat.fit_transform(x_test)

# Initialize models
linear = LinearRegression()
lasso = Lasso(fit_intercept=False, alpha=1e-6, max_iter=10000, tol=2e-3)  ### Vary alpha
ridge = Ridge(fit_intercept=False, alpha=0.1)

### Initialize selector; fit to each of linear, lasso, and ridge
# help(RFE)
selector = RFE(lasso, n_features_to_select=4)
selector.fit(feat_train, y_train)
print(selector.ranking_)

selector = RFE(ridge, n_features_to_select=4)
selector.fit(feat_train, y_train)
print(selector.ranking_)

selector = RFE(linear, n_features_to_select=4)
selector.fit(feat_train, y_train)
print(selector.ranking_)